In [1]:
%pip install davos
import davos

davos.config.suppress_stdout = True

Note: you may need to restart the kernel to use updated packages.


In [2]:
smuggle numpy as np               # pip: numpy==1.24.2
smuggle matplotlib.pyplot as plt  # pip: matplotlib==3.7.0
smuggle pandas as pd              # pip: pandas==1.5.3
smuggle seaborn as sns            # pip: seaborn==0.12.2
smuggle timecorr as tc            # pip: timecorr==0.1.7

from sklearn.decomposition smuggle IncrementalPCA as PCA  # pip: scikit-learn==1.2.1

import os

In [3]:
basedir = os.path.split(os.path.split(os.getcwd())[0])[0]
datadir = os.path.join(basedir, 'data')
figdir = os.path.join(basedir, 'paper', 'figs', 'source')

# Creating synthetic datasets

We create four datasets, exemplifying each of the following scenarios:
  - High informativeness, high compressibility
  - High informativeness, low compressibility
  - Low informativeness, high compressibility
  - Low informativeness, low compressibility

The next cell doesn't actually make sense...maybe use the timecorr synthetic data generator?

Potential idea:
  - Create a multi-subject dataset
  - High informativeness = strong correlations across subjects, each timepoint has a unique pattern
  - High compressibility = highly similar features (e.g., all constant values)

In [42]:
# highly informative datasets have a lot of variability across samples
# highly compressible datasets have lots of repeated structure within samples

buffer = 1000
n_features = 10
n_obs = 20
hi_var = 10
lo_var = 0.001

# alternating ones and zeros
alternating = np.zeros((n_obs, n_features))
alternating[1::2, ::2] = 1
alternating[::2, 1::2] = 1

# highly informative, highly compressible
HI_HC = np.cumsum(hi_var * np.random.normal(size=(n_obs + buffer, n_features)), axis=0)[buffer:, :] + (n_obs * hi_var * alternating)

# highly informative, low compressibility
HI_LC = np.cumsum(hi_var * np.random.normal(size=(n_obs + buffer, n_features)), axis=0)[buffer:, :]

# low informativeness, highly compressible
LI_HC = np.cumsum(lo_var * np.random.normal(size=(n_obs + buffer, n_features)), axis=0)[buffer:, :] + (n_obs * hi_var * alternating)

# low informativeness, low compressibility
LI_LC = np.cumsum(lo_var * np.random.normal(size=(n_obs + buffer, n_features)), axis=0)[buffer:, :]


In [ ]:
# given a dataset, compute peak decoding accuracy